In [175]:
import matplotlib.pyplot as plt
import numpy as np
import math
import urllib.request
import copy

## Question 0

In [176]:
# Iris Data Script
irisUrl = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
iris_raw_data = urllib.request.urlopen(irisUrl)
irisHeaders = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'label']
irisData = np.genfromtxt(iris_raw_data, delimiter=',', dtype=None, usecols=(range(4)))

## Question 1

In [177]:
"""
Function to find distance between two vectors
"""
def LP_distance(x,y,p):
    totalDistance = 0
    for i,j in zip(x,y):
        newDistance = (abs(i-j)**p)
        totalDistance += newDistance
    return (totalDistance**(1/p))

#### Generating random K amount of points for our initial centroids

In [181]:
def make_centroids(x_input,K):
    cluster_k = np.random.randint(0,len(x_input)-1,K)
    initial_centroids = [irisData[i] for i in cluster_k]
    return initial_centroids

In [187]:
"""
cluster_k = np.random.randint(0,len(irisData)-1,3)
initial_centroids = [irisData[i] for i in cluster_k]
"""

initial_centroids = make_centroids(irisData,3)
print(initial_centroids)

[array([ 5.5,  3.5,  1.3,  0.2]), array([ 6.5,  3.2,  5.1,  2. ]), array([ 7.6,  3. ,  6.6,  2.1])]


In [188]:
def k_means_cs171(x_input, K, init_centroids):
    iterations = 1
    p = 2 # Using Euclidean distance
    #cluster_k = list(np.random.randint(0,len(x_input)-1,K))
    cluster_centroids = init_centroids
    change_val = True
    cluster_assignments = []
    while(change_val):
        # Assigning each point to a cluster
        cluster_assignments = []
        for x in range(len(x_input)):
            min_distances = [LP_distance(x_input[x],c,p) for c in cluster_centroids]
            min_index = min_distances.index(min(min_distances))
            cluster_assignments.append(min_index)
        
        # Gather all the points of each cluster and determining if change occurred
        old_cluster_centroids = copy.deepcopy(cluster_centroids)
        for i in range(K):
            cluster_points = [x_input[j] for j in range(len(x_input)) if cluster_assignments[j] == i]
            cluster_centroids[i] = list(np.mean(cluster_points, axis=0))  
            
        if np.allclose(cluster_centroids,old_cluster_centroids,atol=1e-08):
            change_val = False
        print(iterations)
        iterations+=1
        
        
        
    return cluster_assignments,cluster_centroids

In [189]:
cluster_assignments,cluster_centroids = k_means_cs171(irisData, 3, initial_centroids)
print(cluster_assignments)
print(cluster_centroids)

1
2
3
4
5
6
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1]
[[5.0059999999999993, 3.4180000000000006, 1.464, 0.24399999999999991], [5.9016129032258071, 2.7483870967741941, 4.3935483870967751, 1.4338709677419357], [6.8500000000000005, 3.0736842105263151, 5.7421052631578933, 2.0710526315789473]]


## Question 2

## Sources
1. https://docs.scipy.org/doc/numpy/reference/generated/numpy.allclose.html
2. https://docs.python.org/3/library/copy.html
3. 